In [1]:
import pandas as pd
import numpy as np
from autokeras import StructuredDataClassifier
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [6]:
tourney_games = pd.read_csv('../feature_engineering/output/all_games.csv')
reg_season = pd.read_csv('../feature_engineering/output/reg_season_games.csv')


In [3]:
%run ../functions.ipynb

In [10]:
reg_season['t1_margin'] = reg_season['adj_oe'] - reg_season['adj_de']
reg_season['t2_margin'] = reg_season['adj_oe_opponent'] - reg_season['adj_de_opponent']

In [14]:
reg_season['skill_differential'] = abs(reg_season['t1_margin'] - reg_season['t2_margin'])

In [12]:
reg_season['actual_margin'] = reg_season['OffRtg'] - reg_season['DefRtg']

In [16]:
reg_season[['adj_oe', 'adj_de', 'adj_oe_opponent', 'adj_de_opponent',
            't1_margin', 't2_margin', 'skill_differential',
            'OffRtg', 'DefRtg', 'actual_margin'
           ]]

,adj_oe,adj_de,adj_oe_opponent,adj_de_opponent,t1_margin,t2_margin,skill_differential,OffRtg,DefRtg,actual_margin
0,NaN,NaN,122.697869,95.623125,NaN,27.074745,NaN,80.206968,110.924530,-30.717562
1,103.755349,94.893167,99.984008,117.756509,8.862183,-17.772501,26.634684,141.296157,84.385205,56.910952
2,109.867421,97.813673,108.750885,108.875196,12.053748,-0.124311,12.178059,112.708808,102.822070,9.886738
3,107.325578,148.863099,106.902157,102.707406,-41.537522,4.194750,45.732272,95.472569,121.879875,-26.407306
4,150.384969,116.831723,92.722407,112.184261,33.553246,-19.461854,53.015100,150.195948,101.671103,48.524845
...,...,...,...,...,...,...,...,...,...,...
200837,112.160720,130.422972,105.972855,111.003710,-18.262252,-5.030855,13.231397,118.439359,134.930915,-16.491556
200838,84.278685,118.972192,103.444233,105.988815,-34.693506,-2.544583,32.148923,86.960843,119.571159,-32.610316
200839,77.767005,118.464023,105.201585,104.741201,-40.697018,0.460384,41.157402,79.330209,123.572056,-44.241847
200840,107.646010,125.069713,98.430606,103.415845,-17.423703,-4.985238,12.438464,108.131488,121.647924,-13.516436


In [17]:
pd.set_option('max_columns', 1000)
reg_season.head()

,Unnamed: 0,Unnamed: 0.1,TourneyGame,Season,DayNum,Team1,Team2,Team1_score,Team2_score,WLoc,NumOT,Team1_FGM,Team1_FGA,Team1_FGM3,Team1_FGA3,Team1_FTM,Team1_FTA,Team1_OR,Team1_DR,Team1_Ast,Team1_TO,Team1_Stl,Team1_Blk,Team1_PF,Team2_FGM,Team2_FGA,Team2_FGM3,Team2_FGA3,Team2_FTM,Team2_FTA,Team2_OR,Team2_DR,Team2_Ast,Team2_TO,Team2_Stl,Team2_Blk,Team2_PF,Outcome,Loc,TeamID_x,Team1_conf,TeamID_y,Team2_conf,fg_eff,Pos,OffRtg,DefRtg,GameEfficiency,avg_oe,avg_de,opp_avg_oe,opp_avg_de,league_avg_oe,league_avg_de,adj_oe,adj_de,total_games_played,game_num,game_num_adj,adj_oe_opponent,adj_de_opponent,t1_margin,t2_margin,skill_differential,actual_margin
0,0,133,0,2003,19,1102,1257,47,65,H,0,16,40,6,21,9,16,3,17,8,17,2,0,21,23,51,5,17,14,20,13,26,12,14,8,4,23,0,A,1102,mwc,1257,cusa,0.285714,58.5984,80.206968,110.924530,-30.717562,NaN,NaN,NaN,NaN,102.776464,102.833741,NaN,NaN,28,1,1,122.697869,95.623125,NaN,27.074745,NaN,-30.717562
1,1,246,0,2003,22,1102,1391,72,43,H,0,26,46,15,28,5,7,5,22,19,9,6,8,13,15,48,2,7,11,15,12,15,5,11,5,0,12,1,H,1102,mwc,1391,a_sun,0.535714,50.9568,141.296157,84.385205,56.910952,80.206968,110.924530,90.092722,130.269207,102.907100,102.923567,103.755349,94.893167,28,2,1,99.984008,117.756509,8.862183,-17.772501,26.634684,56.910952
2,2,339,0,2003,25,1102,1117,57,52,A,0,16,36,7,20,18,22,3,24,10,10,5,3,20,16,43,6,14,14,21,7,17,8,8,3,0,19,1,A,1102,mwc,1117,sun_belt,0.350000,50.5728,112.708808,102.822070,9.886738,110.751562,97.654868,108.090688,105.683737,103.070205,103.084967,109.867421,97.813673,28,3,1,108.750885,108.875196,12.053748,-0.124311,12.178059,9.886738
3,3,473,0,2003,27,1102,1399,47,60,H,0,15,35,10,19,7,12,1,8,13,12,5,0,20,20,36,2,6,18,20,9,22,13,14,4,1,13,0,A,1102,mwc,1399,ovc,0.526316,49.2288,95.472569,121.879875,-26.407306,111.403978,99.377268,80.437580,90.492278,103.309465,103.319489,107.325578,148.863099,28,4,2,106.902157,102.707406,-41.537522,4.194750,45.732272,-26.407306
4,4,612,0,2003,31,1102,1410,65,44,H,0,23,40,18,33,1,7,5,14,20,7,7,5,16,17,40,3,11,7,9,11,17,7,13,3,0,13,1,H,1102,mwc,1410,ind,0.545455,43.2768,150.195948,101.671103,48.524845,107.421125,105.002920,88.428243,103.802018,103.924944,103.932817,150.384969,116.831723,28,5,2,92.722407,112.184261,33.553246,-19.461854,53.015100,48.524845


In [4]:
tourney_games[tourney_games.type == 0]

,Unnamed: 0,type,ID,Pred,Season,Team1,Team2,Outcome,Team1_conf,Team2_conf,...,t1_adj_margin_6_10,t2_adj_margin_6_10,t1_adj_margin_ratio,t2_adj_margin_ratio,t1_adj_oe_opponent,t1_adj_de_opponent,t2_adj_oe_opponent,t2_adj_de_opponent,t1_opponent_strength,t2_opponent_strength
0,0,0,2003_1421_1411,NaN,2003,1421,1411,1.0,big_south,swac,...,-6.096670,-6.834753,3.782826,0.157641,102.685502,107.542440,99.861495,110.160544,-4.856938,-10.299050
1,1,0,2003_1421_1400,NaN,2003,1421,1400,0.0,big_south,big_twelve,...,-6.096670,20.541063,3.782826,1.486956,102.685502,107.542440,112.110732,100.322368,-4.856938,11.788364
2,2,0,2003_1163_1400,NaN,2003,1163,1400,0.0,big_east,big_twelve,...,14.698067,20.541063,1.791704,1.486956,111.702474,103.099672,112.110732,100.322368,8.602802,11.788364
3,3,0,2003_1277_1400,NaN,2003,1277,1400,0.0,big_ten,big_twelve,...,16.847288,20.541063,1.257998,1.486956,111.184325,100.980257,112.110732,100.322368,10.204068,11.788364
4,4,0,2003_1345_1400,NaN,2003,1345,1400,0.0,big_ten,big_twelve,...,9.400440,20.541063,2.741462,1.486956,111.458320,101.241147,112.110732,100.322368,10.217173,11.788364


In [15]:
feature_list = [
    'seed_diff', 
            't1_adj_margin', 't2_adj_margin',
                't1_final_rank', 't2_final_rank',
                't1_OrdinalRank', 't2_OrdinalRank',
#                 't1_adj_margin_1',
               't1_adj_margin_2', 
                't1_adj_margin_3', 
                't1_adj_margin_4',
                't1_adj_margin_5', 
                't1_adj_margin_6', 
                't1_adj_margin_7',
                   't1_adj_margin_8', 
                't1_adj_margin_9', 
                't1_adj_margin_10',
#                't2_adj_margin_1', 
                't2_adj_margin_2', 
                't2_adj_margin_3',
                   't2_adj_margin_4', 
                    't2_adj_margin_5', 
                't2_adj_margin_6',
                't2_adj_margin_7', 
                't2_adj_margin_8',
                't2_adj_margin_9',
               't2_adj_margin_10',
#             'upset_second_round'
]

In [16]:
df = tourney_games[(tourney_games.Season < 2016)] #This is only for stage 1
#df = df[(df.Season < 2021)] #This is for stage 2
df = df.fillna(0)
X_train = df[feature_list].values
y_train = df['Outcome'].values

X_test = tourney_games[(tourney_games.Season >= 2016) & (tourney_games.Season < 2022)
                          & (tourney_games.type == 0)][feature_list].values
X_test = np.asarray(X_test).astype('float32')
y_test = tourney_games[(tourney_games.Season >= 2016) & (tourney_games.Season < 2022)
                      & (tourney_games.type == 0)]['Outcome'].values
y_test = np.asarray(y_test).astype('float32')


In [17]:

# define the search
search1 = StructuredDataClassifier(max_trials=15)
# perform the search
search1.fit(x=X_train, y=y_train, verbose=100)


Trial 15 Complete [00h 00m 11s]
val_accuracy: 0.7106918096542358

Best val_accuracy So Far: 0.7327044010162354
Total elapsed time: 00h 01m 29s
INFO:tensorflow:Oracle triggered exit
53/53 [==============================] - 1s 2ms/step - loss: 0.6263 - accuracy: 0.6458


2022-03-05 21:13:23.165735: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./structured_data_classifier/best_model/assets


In [18]:
# evaluate the model
loss, acc = search1.evaluate(X_test, y_test, verbose=1000)


In [19]:
loss

0.5823287963867188

In [25]:
model = search1.export_model()
y_pred_keras = model.predict(X_test)
ll = log_loss(y_test, y_pred_keras), accuracy_score(y_test, np.where(y_pred_keras > .5, 1, 0))
acc = accuracy_score(y_test, np.where(y_pred_keras > .5, 1, 0))

print(acc, ll)

0.6856287425149701 (0.5823287842739485, 0.6856287425149701)


In [30]:

clf = LogisticRegression(random_state = 0, C = 0.0001, penalty = 'l2')
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)
ll = log_loss(y_test, y_pred)
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print(acc, ll)

0.688622754491018 0.575894189192754


In [ ]:
# add least for now, autokeras isn't helping over my Logreg
